## Model Inference

In [3]:
import os,torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 출력
print('Device:', device)  # 출력결과: cuda
print('Count of using GPUs:', torch.cuda.device_count())   #출력결과: 1 (GPU #2 한개 사용하므로)
print('Current cuda device:', torch.cuda.current_device())  # 출력결과: 0 (수정 필요)

Device: cuda
Count of using GPUs: 4
Current cuda device: 0


In [6]:
new_model= "Fine tuning Edentns-DataVortexS with train_combined_doubleQ"
bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
tokenizer = AutoTokenizer.from_pretrained(new_model, padding_side="left")
model = AutoModelForCausalLM.from_pretrained(
        new_model,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )
model.config.use_cache = True

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [11]:
import pandas as pd
from tqdm import tqdm
test = pd.read_csv('./test.csv')

batch_size=8

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, 
                temperature=0.1,
                # top_k=1,
                # top_p=0.9,
                # repetition_penalty=1.2,
                do_sample=True,
                # num_return_sequences=1,
                #max_length=300,
                max_new_tokens=300,
                batch_size=batch_size,
               )
preds = []
for i in tqdm(range(0, len(test), batch_size)):
    batch_questions = test['질문'][i:i+batch_size].tolist()
    batch_prompts = [f"<s> Question: {q} Answer: </s>" for q in batch_questions]
    batch_results = pipe(batch_prompts)

    for batch_item in batch_results:
        for result in batch_item:  # 배치 결과 내의 각 결과 아이템을 순회
            parsed_ans = result['generated_text']
            preds.append(parsed_ans)

# 결과 출력
for i, (question, ans) in enumerate(zip(test['질문'], preds)):
    print(f"[ {i}번 ] 질문: {question}")
    print(f"[ 답변 ] {ans}\n")

print("총 생성된 답변 개수 : ", len(preds))

 59%|████████████████████████████████████████████████▏                                 | 10/17 [47:59<35:30, 304.41s/it]/shared/home/qhdrl527/anaconda3/envs/mllm/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
100%|████████████████████████████████████████████████████████████████████████████████| 17/17 [1:21:18<00:00, 286.96s/it]

[ 0번 ] 질문: 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
[ 답변 ] <s> Question: 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요? Answer: </s> 방청 페인트의 종류로는 방청 페인트, 방청 도료, 방청 도장재 등이 있습니다. 이러한 제품들은 각각의 특징과 용도에 따라 선택되며, 사용 환경과 목적에 맞게 선택해야 합니다. 그리고 원목사이딩의 단점은 주로 비교적 높은 가격과 유지보수가 필요하다는 점입니다. 또한, 원목사이딩은 습기에 취약하여 변형이 발생할 수 있으며, 외부 환경에 따라 변색이 발생

[ 1번 ] 질문: 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
[ 답변 ] <s> Question: 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요? Answer: </s> 도배지에 녹은 자국이 발생하는 원인은 주로 높은 습도와 높은 온도입니다. 습기가 높을 때 도배지에 녹은 자국이 생길 수 있는데, 이는 도배지가 습기를 흡수하여 변형되거나 녹은 자국이 생길 수 있기 때문입니다. 또한, 높은 온도로 인해 도배지가 팽창하거나 수축하면서 녹은 자국이 생길 수 있습니다. 이러한 원인으로 인해 도배

[ 2번 ] 질문: 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
[ 답변 ] <s> Question: 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요? Answer: </s> 큐블럭은 시공이 어렵고 시공 오류로 인해 결로가 발생할 수 있는 단점이 있습니다. 또한, 큐블럭은 시공 후 충격에 약해 파손될 수 있는 점도 고려해야 합니다. 그리고 압출법 단열판은 시공이 간편하고 시공 후 충격에 강하며 내화성이 뛰어나다는 장점이 있습니다. 또한, 압출법 단열판은 열전도율이 낮아 단열성능이 뛰

[ 3

In [15]:
submit = pd.DataFrame({"답변": preds})
submit.to_csv(f'./{new_model}_korean.csv', index=False)

In [19]:
cleaned_preds = [pred.split('</s>')[1] for pred in preds]

In [26]:
cleaned_preds[:2]

[' 방청 페인트의 종류로는 방청 페인트, 방청 도료, 방청 도장재 등이 있습니다. 이러한 제품들은 각각의 특징과 용도에 따라 선택되며, 사용 환경과 목적에 맞게 선택해야 합니다. 그리고 원목사이딩의 단점은 주로 비교적 높은 가격과 유지보수가 필요하다는 점입니다. 또한, 원목사이딩은 습기에 취약하여 변형이 발생할 수 있으며, 외부 환경에 따라 변색이 발생',
 ' 도배지에 녹은 자국이 발생하는 원인은 주로 높은 습도와 높은 온도입니다. 습기가 높을 때 도배지에 녹은 자국이 생길 수 있는데, 이는 도배지가 습기를 흡수하여 변형되거나 녹은 자국이 생길 수 있기 때문입니다. 또한, 높은 온도로 인해 도배지가 팽창하거나 수축하면서 녹은 자국이 생길 수 있습니다. 이러한 원인으로 인해 도배']

In [27]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(cleaned_preds)
pred_embeddings.shape

(130, 512)

In [28]:
submit = pd.read_csv('./sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()

/tmp/ipykernel_1027226/1828452193.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.08310204  0.01070711 -0.00524251  0.00881512 -0.00158785  0.02720487
 -0.01335036  0.0014422  -0.02882941  0.00852075  0.02237136  0.0058351
 -0.00316639 -0.04161518  0.012243    0.04705796  0.01739692 -0.01020982
  0.01612945 -0.01253391 -0.01208979  0.01669472 -0.03293391  0.02354943
  0.00938061 -0.00981834  0.03737142 -0.02130329 -0.01102856 -0.01667357
 -0.00598879 -0.00558314 -0.03211967  0.00115653 -0.01282257 -0.00106181
  0.0009664   0.00733454 -0.01010328  0.03522259  0.00798082  0.01012336
  0.02491123  0.02676681  0.00285239  0.02959864 -0.00829805 -0.0422413
 -0.00478921 -0.06221235  0.00137794  0.01101704  0.00847057 -0.03440005
  0.00867071 -0.04338013  0.00670969 -0.01108245  0.00602367  0.03480835
 -0.01332578 -0.02405409  0.00874598 -0.01209424 -0.00364368  0.01497657
  0.02475812  0.03521512  0.01552229 -0.0

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.083102,0.104993,0.022009,-0.018183,0.090990,-0.011164,0.035533,0.070488,0.034887,...,0.006397,-0.051809,0.006574,-0.047566,-0.029303,0.048534,0.044054,0.019990,-0.007651,0.032923
1,TEST_001,0.010707,0.022885,-0.023668,-0.003932,0.095467,-0.022362,0.005083,-0.034604,-0.026612,...,-0.067001,-0.036924,0.016512,-0.018420,-0.028636,0.027762,-0.028728,-0.008130,-0.015204,0.031079
2,TEST_002,-0.005243,-0.044467,-0.063820,-0.012991,0.133594,-0.059750,-0.015322,0.001905,0.048166,...,-0.048262,-0.016249,0.035398,-0.038800,0.031581,0.020811,-0.064008,-0.025736,-0.043605,0.065216
3,TEST_003,0.008815,0.036128,-0.017902,0.050348,0.068351,-0.076165,-0.043005,-0.039296,-0.005987,...,-0.026350,-0.019227,0.038792,-0.052758,-0.003442,0.047432,-0.037210,-0.067973,-0.019085,0.054317
4,TEST_004,-0.001588,0.033933,-0.006947,-0.012813,0.116697,-0.006842,0.078884,0.022196,-0.001912,...,-0.016674,-0.035814,0.033550,0.019780,-0.024662,0.037790,-0.008059,0.055282,0.002007,0.030674


In [29]:
# 리더보드 제출을 위한 csv파일 생성
submit.to_csv(f'./{new_model}.csv', index=False)